In [ ]:
# !git clone --branch Distributed-Data-Parallel https://github.com/TAYDOVAT/Cuoi_Ki_DL.git
# %cd /kaggle/working/Cuoi_Ki_DL

In [ ]:
from copy import deepcopy
from configs import CFG

cfg = deepcopy(CFG)


In [ ]:
# Config override here (dataset)
from pathlib import Path

# Fixed working directory for relative paths
cwd = Path('working/Cuoi_Ki_DL')
if not cwd.exists():
    # Fallback when running from repo root or other cwd
    alt = Path.cwd() / 'working' / 'Cuoi_Ki_DL'
    if alt.exists():
        cwd = alt
    else:
        # Last-resort: use current working directory
        cwd = Path.cwd()

data_root = cwd / '..' / '..' / 'input' / 'datasets' / 'tyantran' / 'anh-ve-tinh-2' / 'Anh_ve_tinh_2'

cfg['paths']['test_lr'] = str(data_root / 'test' / 'test_lr')
cfg['paths']['test_hr'] = str(data_root / 'test' / 'test_hr')

print('CWD:', cwd)
print('Test LR:', cfg['paths']['test_lr'])
print('Test HR:', cfg['paths']['test_hr'])


In [ ]:
# Weights config (relative paths)
from pathlib import Path

# Fixed working directory for relative paths
cwd = Path('working/Cuoi_Ki_DL')
if not cwd.exists():
    # Fallback when running from repo root or other cwd
    alt = Path.cwd() / 'working' / 'Cuoi_Ki_DL'
    if alt.exists():
        cwd = alt
    else:
        # Last-resort: use current working directory
        cwd = Path.cwd()

weights_dir = cwd / '..' / '..' / 'input' / 'datasets' / 'tyantran' / 'srresnet' / 'srresnet'
# Choose one: srresnet_lpips_epoch_20.pth | srresnet_l1_epoch_25.pth | srresnet_ssim_epoch_69.pth
weights_name = 'srresnet_l1_epoch_25.pth'
weights_path = weights_dir / weights_name

print('Weights:', weights_path)


In [ ]:
# Evaluation config (edit all evaluation-related settings here)
from pathlib import Path

# Fixed working directory for relative paths
cwd = Path('working/Cuoi_Ki_DL')
if not cwd.exists():
    # Fallback when running from repo root or other cwd
    alt = Path.cwd() / 'working' / 'Cuoi_Ki_DL'
    if alt.exists():
        cwd = alt
    else:
        # Last-resort: use current working directory
        cwd = Path.cwd()

eval_cfg = {
    'batch_size': 1,
    'num_workers': 2,
    'use_amp': True,
    'device': 'cuda',
    'compute_psnr': True,
    'compute_ssim': True,
    'compute_lpips': True,
    'lpips_net': 'vgg',
    'save_sr': True,
    'save_dir': str(cwd / '..' / '..' / 'outputs' / 'srresnet_test'),
    'max_images': None,
}

print('Eval config:', eval_cfg)


In [ ]:
# !pip -q install lpips


In [ ]:
import torch
from model import SRResNet
from data import build_loader
from metrics import psnr, ssim

device = torch.device(eval_cfg['device'] if torch.cuda.is_available() and eval_cfg['device'] == 'cuda' else 'cpu')
print('Device:', device)

model = SRResNet(scale=cfg['scale'])
state = torch.load(weights_path, map_location='cpu')
if isinstance(state, dict):
    if 'model_state_dict' in state:
        state = state['model_state_dict']
    elif 'generator_state_dict' in state:
        state = state['generator_state_dict']
model.load_state_dict(state)
model.to(device)
model.eval()
print('Loaded weights OK')


In [ ]:
test_lr = cfg['paths']['test_lr']
test_hr = cfg['paths']['test_hr']

dataset, loader = build_loader(
    lr_dir=test_lr,
    hr_dir=test_hr,
    scale=cfg['scale'],
    hr_crop=cfg['hr_crop'],
    batch_size=eval_cfg['batch_size'],
    num_workers=eval_cfg['num_workers'],
    train=False,
)

print('Test samples:', len(dataset))


In [ ]:
import os
from pathlib import Path
from torchvision.transforms import functional as TF

try:
    import lpips
except Exception as e:
    lpips = None
    print('LPIPS not available:', e)

lpips_metric = None
if eval_cfg['compute_lpips'] and lpips is not None:
    lpips_metric = lpips.LPIPS(net=eval_cfg['lpips_net']).to(device)
    lpips_metric.eval()

save_dir = Path(eval_cfg['save_dir'])
if eval_cfg['save_sr']:
    save_dir.mkdir(parents=True, exist_ok=True)

total_psnr = 0.0
total_ssim = 0.0
total_lpips = 0.0
metric_count = 0
img_index = 0

max_images = eval_cfg['max_images']

device_type = device.type if hasattr(device, 'type') else 'cpu'

with torch.no_grad():
    for i, (lr, hr) in enumerate(loader):
        if max_images is not None and img_index >= max_images:
            break

        lr = lr.to(device, non_blocking=True)
        hr = hr.to(device, non_blocking=True)

        with torch.amp.autocast(device_type=device_type, enabled=eval_cfg['use_amp']):
            sr = model(lr)

        sr_clip = sr.clamp(0.0, 1.0)
        batch_size = lr.size(0)

        if eval_cfg['compute_psnr']:
            total_psnr += psnr(sr_clip.float(), hr.float()) * batch_size
        if eval_cfg['compute_ssim']:
            total_ssim += ssim(sr_clip.float(), hr.float()) * batch_size
        if lpips_metric is not None:
            sr_norm = sr_clip * 2.0 - 1.0
            hr_norm = hr * 2.0 - 1.0
            total_lpips += lpips_metric(sr_norm, hr_norm).mean().item() * batch_size

        metric_count += batch_size

        # Save SR images
        if eval_cfg['save_sr']:
            batch = sr_clip.cpu()
            for b in range(batch.size(0)):
                if max_images is not None and img_index >= max_images:
                    break
                lr_path, hr_path = dataset.pairs[img_index]
                stem = Path(lr_path).stem
                out_path = save_dir / f'{stem}_SR.png'
                img = TF.to_pil_image(batch[b])
                img.save(out_path)
                img_index += 1
        else:
            img_index += batch_size

if metric_count == 0:
    raise RuntimeError('No samples processed.')

avg_psnr = total_psnr / metric_count if eval_cfg['compute_psnr'] else 0.0
avg_ssim = total_ssim / metric_count if eval_cfg['compute_ssim'] else 0.0
avg_lpips = total_lpips / metric_count if lpips_metric is not None else 0.0

print(f'PSNR: {avg_psnr:.4f}')
print(f'SSIM: {avg_ssim:.4f}')
print(f'LPIPS: {avg_lpips:.4f}')
print('Saved to:', save_dir if eval_cfg['save_sr'] else 'N/A')
